In [1]:
import pandas as pd 
import numpy as np 
import os 
import sys 
import re 
import json
import ast

In [2]:
dir = "../../data/conv-trs/eval/coverage/"

llama = pd.read_csv(f"{dir}llama_groundedness.csv")

In [3]:
llama.head()

,qg_model,config_id,filters,query_v,query_v_matches,query_p0,query_p0_matches,query_p1,query_p1_matches
0,llama,c_p_0_pop_low_easy,"{'popularity': 'low', 'month': 'February'}","""Less crowded European cities to visit in Febr...","Matches: 2\nExplanation: ""Less crowded"" matche...","""European cities with ice hockey facilities, l...","Matches: 2\nExplanation: ""low popularity"" matc...",Which European cities have ice hockey arenas a...,"Matches: 0\nExplanation: Neither ""low"" popular..."
1,llama,c_p_0_pop_low_medium,"{'popularity': 'low', 'budget': 'low', 'intere...","""Low budget European city breaks with old tree...","Matches: 2\nExplanation:\n* ""Low budget"" match...","""European cities with low budget options and o...",Matches: 3\nExplanation:\n* `budget: low` matc...,Which European cities have ice hockey arenas a...,"Matches: 2\nExplanation: The query mentions ""a..."
2,llama,c_p_0_pop_low_hard,"{'popularity': 'low', 'budget': 'high', 'month...","""Unconventional European destinations with mus...",Matches: 3\nExplanation:\n* **budget: 'high'**...,"""European cities with low popularity, high bud...","Matches: 4\nExplanation:\n* ""low popularity"" m...",Which European cities have ice hockey arenas a...,"Matches: 1\nExplanation: The query mentions ""a..."
3,llama,c_p_0_pop_low_sustainable,"{'popularity': 'low', 'budget': 'low', 'month'...","""Low budget destinations in Europe with good a...","Matches: 3\nExplanation: The query mentions ""l...","""Low budget European cities with great air qua...",Matches: 4\nExplanation:\n* `budget: low` matc...,Which European cities have ice hockey arenas a...,"Matches: 2\nExplanation: ""affordable accommoda..."
4,llama,c_p_0_pop_medium_easy,"{'popularity': 'medium', 'month': 'January'}","""Medium-sized European cities to visit in Janu...","Matches: 2\nExplanation: ""Medium-sized"" matche...","""European cities with ice hockey facilities, m...","Matches: 2\nExplanation: ""medium popularity"" m...",Which European cities have ice hockey arenas a...,Matches: 0\nExplanation: Neither of the filter...


In [4]:
for method in ['v', 'p0', 'p1']:
    llama[f'matches_{method}'] = llama[f'query_{method}_matches'].str.extract(r'Matches:\s*([\d]+)', expand=False).astype(float).astype('Int64')
    llama[f'explanation_{method}'] = llama[f'query_{method}_matches'].str.extract(r'Explanation:\s*(.*)', flags=re.DOTALL)

In [5]:
for method in ['v', 'p0', 'p1']:
    llama[f"normalized_matches_{method}"] = llama.apply(lambda row: row[f'matches_{method}'] / len(ast.literal_eval(row['filters'])), axis=1)

In [6]:
llama.head()

,qg_model,config_id,filters,query_v,query_v_matches,query_p0,query_p0_matches,query_p1,query_p1_matches,matches_v,explanation_v,matches_p0,explanation_p0,matches_p1,explanation_p1,normalized_matches_v,normalized_matches_p0,normalized_matches_p1
0,llama,c_p_0_pop_low_easy,"{'popularity': 'low', 'month': 'February'}","""Less crowded European cities to visit in Febr...","Matches: 2\nExplanation: ""Less crowded"" matche...","""European cities with ice hockey facilities, l...","Matches: 2\nExplanation: ""low popularity"" matc...",Which European cities have ice hockey arenas a...,"Matches: 0\nExplanation: Neither ""low"" popular...",2,"""Less crowded"" matches the 'popularity': 'low'...",2,"""low popularity"" matches the 'popularity': 'lo...",0,"Neither ""low"" popularity nor ""February"" are ex...",1.000000,1.0,0.000000
1,llama,c_p_0_pop_low_medium,"{'popularity': 'low', 'budget': 'low', 'intere...","""Low budget European city breaks with old tree...","Matches: 2\nExplanation:\n* ""Low budget"" match...","""European cities with low budget options and o...",Matches: 3\nExplanation:\n* `budget: low` matc...,Which European cities have ice hockey arenas a...,"Matches: 2\nExplanation: The query mentions ""a...",2,"* ""Low budget"" matches the 'budget': 'low' fil...",3,"* `budget: low` matches ""low budget options"" i...",2,"The query mentions ""affordable accommodations,...",0.666667,1.0,0.666667
2,llama,c_p_0_pop_low_hard,"{'popularity': 'low', 'budget': 'high', 'month...","""Unconventional European destinations with mus...",Matches: 3\nExplanation:\n* **budget: 'high'**...,"""European cities with low popularity, high bud...","Matches: 4\nExplanation:\n* ""low popularity"" m...",Which European cities have ice hockey arenas a...,"Matches: 1\nExplanation: The query mentions ""a...",3,"* **budget: 'high'**: The query mentions ""will...",4,"* ""low popularity"" matches the `popularity: lo...",1,"The query mentions ""affordable accommodations,...",0.750000,1.0,0.250000
3,llama,c_p_0_pop_low_sustainable,"{'popularity': 'low', 'budget': 'low', 'month'...","""Low budget destinations in Europe with good a...","Matches: 3\nExplanation: The query mentions ""l...","""Low budget European cities with great air qua...",Matches: 4\nExplanation:\n* `budget: low` matc...,Which European cities have ice hockey arenas a...,"Matches: 2\nExplanation: ""affordable accommoda...",3,"The query mentions ""low budget"" matching the '...",4,"* `budget: low` matches ""Low budget"" in the qu...",2,"""affordable accommodations"" matches the 'budge...",0.750000,1.0,0.500000
4,llama,c_p_0_pop_medium_easy,"{'popularity': 'medium', 'month': 'January'}","""Medium-sized European cities to visit in Janu...","Matches: 2\nExplanation: ""Medium-sized"" matche...","""European cities with ice hockey facilities, m...","Matches: 2\nExplanation: ""medium popularity"" m...",Which European cities have ice hockey arenas a...,Matches: 0\nExplanation: Neither of the filter...,2,"""Medium-sized"" matches the 'popularity':'mediu...",2,"""medium popularity"" matches the `popularity: m...",0,Neither of the filters are present in the quer...,1.000000,1.0,0.000000


In [7]:
# Average recall
for method in ['v', 'p0', 'p1']:    
    print(f"Average Groundedness Recall for method {method} is {round(llama[f'normalized_matches_{method}'].mean(), 3)}")

Average Groundedness Recall for method v is 0.801
Average Groundedness Recall for method p0 is 0.676
Average Groundedness Recall for method p1 is 0.093


In [8]:
p1 = llama[llama['matches_p1'] == 0]['query_p1'].tolist()

In [9]:
len(p1)/len(llama)

0.7224152910512598

In [10]:
gemini = pd.read_csv(f"{dir}gemini_groundedness.csv")

In [11]:
for method in ['v', 'p0', 'p1']:
    gemini[f'matches_{method}'] = gemini[f'query_{method}_matches'].str.extract(r'Matches:\s*([\d]+)', expand=False).astype(float).astype('Int64')
    gemini[f'explanation_{method}'] = gemini[f'query_{method}_matches'].str.extract(r'Explanation:\s*(.*)', flags=re.DOTALL)
    gemini[f"normalized_matches_{method}"] = gemini.apply(lambda row: row[f'matches_{method}'] / len(ast.literal_eval(row['filters'])), axis=1)

In [12]:
# Average recall
for method in ['v', 'p0', 'p1']:    
    print(f"Average Groundedness Recall for method {method} is {round(gemini[f'normalized_matches_{method}'].mean(),3)}")

Average Groundedness Recall for method v is 0.727
Average Groundedness Recall for method p0 is 0.611
Average Groundedness Recall for method p1 is 0.056


In [29]:
llama = pd.read_csv(f"{dir}llama_groundedness_gpt.csv")

for method in ['v', 'p0', 'p1']:
    llama[f'matches_{method}'] = llama[f'query_{method}_matches'].str.extract(r'Matches:\s*([\d]+)', expand=False).astype(float).astype('Int64')
    llama[f'explanation_{method}'] = llama[f'query_{method}_matches'].str.extract(r'Explanation:\s*(.*)', flags=re.DOTALL)
    llama[f"normalized_matches_{method}"] = llama.apply(lambda row: row[f'matches_{method}'] / len(ast.literal_eval(row['filters'])), axis=1)
    print(f"Average Groundedness Recall for method {method} is {round(llama[f'normalized_matches_{method}'].mean(),3)}")

Average Groundedness Recall for method v is 0.759
Average Groundedness Recall for method p0 is 0.644
Average Groundedness Recall for method p1 is 0.043


In [32]:
gemini = pd.read_csv(f"{dir}gemini_groundedness_gpt.csv")

for method in ['v', 'p0', 'p1']:
    gemini[f'matches_{method}'] = gemini[f'query_{method}_matches'].str.extract(r'Matches:\s*([\d]+)', expand=False).astype(float).astype('Int64')
    gemini[f'explanation_{method}'] = gemini[f'query_{method}_matches'].str.extract(r'Explanation:\s*(.*)', flags=re.DOTALL)
    gemini[f"normalized_matches_{method}"] = gemini.apply(lambda row: row[f'matches_{method}'] / len(ast.literal_eval(row['filters'])), axis=1)
    print(f"Average Groundedness Recall for method {method} is {round(gemini[f'normalized_matches_{method}'].mean(),3)}")

Average Groundedness Recall for method v is 0.717
Average Groundedness Recall for method p0 is 0.595
Average Groundedness Recall for method p1 is 0.042
